In [ ]:
from sys import path

from pandas import DataFrame

path.append("../../../")
from componentes.pipeline.pipeline import Pipeline
from componentes.artefato.arquivo import Arquivo
from componentes.container.container_gold import ContainerGold
from componentes.container.categoria_de_armazenamento import \
    CategoriaDeArmazenamento
from notebooks.silver.paic.dataframe_paic_silver import DataFramePAICSilver

In [ ]:
class DimensaoCategoriaEmpresaSilverParaGold(Pipeline):
    """
    Pipeline de dados responsável por executar o processamento de artefatos
    de categoria silver para gold gerando a dimensao categoria empresa
    """
    def _extrair_dados_da_fonte(self) -> DataFrame:
        """
        Extraí os dados usando o objeto DataFramePAICSilver
        """
        self._logger.log_mensagem(mensagem=f"Criando a tabela PAIC silver")
        return DataFramePAICSilver().criar_dataframe()

    def _transformar_dados(self, dataframe: DataFrame) -> DataFrame:
        dataframe = self._selecionar_e_renomear_colunas(dataframe=dataframe)
        dataframe = self._criar_categorias_distintas(dataframe=dataframe)
        dataframe = self._criar_id_categoria(dataframe=dataframe)
        return dataframe

    def _selecionar_e_renomear_colunas(self, dataframe: DataFrame) -> DataFrame:
        """
        Seleciona as colunas necessárias do DataFrame PAIC silver e as renomeia
        de acordo com a modelagem definida
        """
        self._logger.log_mensagem(mensagem="Selecionando e renomenado colunas")
        return (
            dataframe[["categoria_empresa"]]
            .rename(columns={"categoria_empresa": "nome"}))

    def _criar_categorias_distintas(self,dataframe: DataFrame) -> DataFrame:
        """
        Cria um novo DataFrame com a remoção de linhas duplicadas
        """
        return dataframe.drop_duplicates()

    def _criar_id_categoria(self, dataframe: DataFrame) -> DataFrame:
        """
        Criar um novo identificador único para dimensão
        """
        self._logger.log_mensagem(mensagem="Criando coluna id_categoria")
        dataframe["id_categoria"] = dataframe.reset_index().index + 1
        return dataframe

In [ ]:
artefato_de_saida = Arquivo(
    nome="dimensao_categoria_empresa",
    categoria_de_armazenamento=CategoriaDeArmazenamento.GOLD,
    fonte="dimensao")
container_de_saida = ContainerGold()
pipeline = DimensaoCategoriaEmpresaSilverParaGold(
    artefato_de_entrada=None,
    container_de_entrada=None,
    artefato_de_saida=artefato_de_saida,
    container_de_saida=container_de_saida)
pipeline.executar_etl()